In [86]:
import pandas as pd

In [87]:
urls = pd.read_csv("./naver_KBS_url.csv")

In [90]:
urls.reset_index(drop = True, inplace=True)
urls = urls.iloc[:,[1,2]]

In [144]:
urls_lst = urls.iloc[:,0]
vid_lst = urls.iloc[:,1]

In [167]:
len(urls_lst)

28423

In [118]:
url = urls_lst[2]
print(url)
oid = re.findall('article/([^\*]*)/', str(url))[0]
aid = re.findall(f'{oid}/([^\*]*)[?]', str(url))[0]
sid = url[-3:]
print(oid, aid, sid)

https://n.news.naver.com/mnews/article/056/0000011174?sid=102
056 0000011174 102


In [157]:
# 댓글을 달 빈 리스트를 생성합니다.
body_lst = []
reply_count_lst = []
uid_lst = []
like_lst = []
dislike_lst = []
c_time_lst= []
cid_lst = []
pid_lst = []



# 라이브러리를 로드합니다.
from bs4 import BeautifulSoup
import requests
import re
import sys
import pprint
from tqdm import tqdm

total_df = pd.DataFrame()
 
for url, vid in tqdm(zip(urls_lst, vid_lst)):

    body_lst = []
    reply_count_lst = []
    uid_lst = []
    like_lst = []
    dislike_lst = []
    c_time_lst= []
    cid_lst = []
    pid_lst = []
    try:
        oid = re.findall('article/([^\*]*)/', str(url))[0]
        aid = re.findall(f'{oid}/([^\*]*)[?]', str(url))[0]
    except:
        print(url)
        continue

    sid = url[-3:]
    page = 1
    header = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        "referer": url,
    }
    
    while True:
        c_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" + oid + "%2C" + aid + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=" + str(
            page) + "&refresh=false&sort=FAVORITE"
        # 파싱하는 단계입니다.
        r = requests.get(c_url, headers=header)
        cont = BeautifulSoup(r.content, "html.parser")
        total_comm = str(cont).split('comment":')[1].split(",")[0]
    
        body = re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
        reply_count = re.findall('"replyCount":([^\*]*),"replyAllCount"', str(cont))
        uid = re.findall('"userIdNo":([^\*]*),"exposedUserIp"', str(cont))
        like = re.findall('sympathyCount":([^\*]*),"antipathyCount"', str(cont))
        dislike = re.findall('"antipathyCount":([^\*]*),"hideReplyButton"', str(cont))
        c_time = re.findall('"modTime":([^\*]*),"modTimeGmt"', str(cont))
        cid = re.findall('commentNo":"([^\*]*)","parentCommentNo"', str(cont))
        pid = re.findall('"parentCommentNo":"([^\*]*)","replyLevel"', str(cont))

        # 댓글을 리스트에 중첩합니다.
        body_lst.append(body)
        reply_count_lst.append(reply_count)
        uid_lst.append(uid)
        like_lst.append(like)
        dislike_lst.append(dislike)
        cid_lst.append(cid)
        pid_lst.append(pid)
        c_time_lst.append(c_time)
        # 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.
        if int(total_comm) <= ((page) * 20):
            break
        else:
            page += 1
    
    
    # 여러 리스트들을 하나로 묶어 주는 함수입니다.
    def flatten(l):
        flatList = []
        for elem in l:
            # if an element of a list is a list
            # iterate over this list and add elements to flatList
            if type(elem) == list:
                for e in elem:
                    flatList.append(e)
            else:
                flatList.append(elem)
        return flatList
    
    
    # 리스트 결과입니다.
    
    allCommetns = flatten(body_lst)
    allreply_count = flatten(reply_count_lst)
    alluid = flatten(uid_lst)
    alllike = flatten(like_lst)
    alldislike = flatten(dislike_lst)
    allctime = flatten(c_time_lst)
    allcid = flatten(cid_lst)
    allpid = flatten(pid_lst)

    url_df = pd.DataFrame([allCommetns, allreply_count,alluid, alllike, alldislike, allctime, allcid, allpid]).T
    url_df['sid'] = sid
    time = re.findall('"date":"([^\*]*)"[}]', str(cont))[0]
    url_df['time'] = time
    url_df['vid'] = vid

    url_df.columns = ['comment', 'reply', 'author', 'like', 'dislike', 'c_time', 'cid', 'pid', 'sid', 'time', 'vid']

    total_df = pd.concat([total_df, url_df])

#print(allCommetns)
# print(len(allCommetns))
# print(len(allreply_count))
# print(len(alluid))
# print(len(alllike))
# print(len(alldislike))
# print(len(allctime))
# print(len(allcid))
# print(len(allpid))

In [158]:
total_df.groupby('vid').size()

vid
--VCE5EYFY8      9
--_do-VV13g      1
--wikkV2QoY     40
--zPmF9nGi0      1
-00l6qnrJn8      4
              ... 
zyar6J3oatw      7
zz4V8AW3SHE    900
zzb79pkt--Y    180
zzpJuM3N1k0      2
zzuPDmJ3LIo      2
Length: 17126, dtype: int64

In [161]:
total_df.to_pickle("./naver_KBS_comments.pkl")

In [162]:
total_df

,comment,reply,author,like,dislike,c_time,cid,pid,sid,time,vid
0,"""좋은기사 잘읽었습니다^-^""",0,"""2LtTb""",0,0,"""2016-04-03T10:05:00+0900""",580891842,580891842,103,2022-09-26T07:44:39+0000,ygvHWqa4GbU
0,"""도박장을 도박장이라고 말하는게 어찌 선동이난 알바야‥! 네눈에 그리 보인다믄 넌 ...",0,"""1Q5id""",5,0,"""2014-07-29T12:24:51+0900""",12163942,12163942,102,2022-09-26T07:44:39+0000,P988js8z2nk
1,"""작년 기사지만 화상경마장의 모습은 변함이 없이 한결같다. 화상경마를 향한 지고지...",0,"""pl8m""",2,0,"""2014-07-29T13:19:56+0900""",12163945,12163945,102,2022-09-26T07:44:39+0000,P988js8z2nk
2,"""농식품부가 문제군요.관련 공무원들 징계해야 되는데, 문제발단은 돈 먹은 농식품부.""",0,"""1N6PL""",0,0,"""2014-07-29T13:18:53+0900""",12163944,12163944,102,2022-09-26T07:44:39+0000,P988js8z2nk
3,"""기사를 쓰는거냐? 선동을 하는거냐? 개나소나 기자라고... 저건 팩트를 전달하겠다...",2,"""2xLak""",7,10,"""2013-08-31T11:09:29+0900""",12163941,12163941,102,2022-09-26T07:44:39+0000,P988js8z2nk
...,...,...,...,...,...,...,...,...,...,...,...
13,"""코로나 백신 페기도 기사화 해주시죠..""",0,"""3CxLG""",0,0,"""2022-08-31T10:20:55+0900""",767965981585703361,767965981585703361,102,2022-09-26T13:58:34+0000,Vm2WPiKsIw0
14,"""그래 좋은 정책👍""",0,"""2k0ZP""",1,2,"""2022-08-31T10:15:56+0900""",767965660922773957,767965660922773957,102,2022-09-26T13:58:34+0000,Vm2WPiKsIw0
15,"""정신나간 개돼지들이 추석연휴때 또 줄서서 PCR 유르르 받겠네.""",1,"""1kS72""",0,1,"""2022-08-31T09:57:01+0900""",767964442628456458,767964442628456458,102,2022-09-26T13:58:34+0000,Vm2WPiKsIw0
16,"""백신팔이 가동한 윤깡통을 지지하는 대깨윤들은 정부에서 4분기에 준다는 개량백신 다...",0,"""100n7""",3,5,"""2022-08-31T09:59:27+0900""",767964598623011043,767964598623011043,102,2022-09-26T13:58:34+0000,Vm2WPiKsIw0


In [164]:
total_df.to_csv("./naver_KBS_comments.csv")

In [3]:
import pandas as pd
total_df = pd.read_csv("./naver_KBS_comments.csv")

In [4]:
total_df

,Unnamed: 0,comment,reply,author,like,dislike,c_time,cid,pid,sid,time,vid
0,0,"""좋은기사 잘읽었습니다^-^""",0,"""2LtTb""",0,0,"""2016-04-03T10:05:00+0900""",580891842,580891842,103,2022-09-26T07:44:39+0000,ygvHWqa4GbU
1,0,"""도박장을 도박장이라고 말하는게 어찌 선동이난 알바야‥! 네눈에 그리 보인다믄 넌 ...",0,"""1Q5id""",5,0,"""2014-07-29T12:24:51+0900""",12163942,12163942,102,2022-09-26T07:44:39+0000,P988js8z2nk
2,1,"""작년 기사지만 화상경마장의 모습은 변함이 없이 한결같다. 화상경마를 향한 지고지...",0,"""pl8m""",2,0,"""2014-07-29T13:19:56+0900""",12163945,12163945,102,2022-09-26T07:44:39+0000,P988js8z2nk
3,2,"""농식품부가 문제군요.관련 공무원들 징계해야 되는데, 문제발단은 돈 먹은 농식품부.""",0,"""1N6PL""",0,0,"""2014-07-29T13:18:53+0900""",12163944,12163944,102,2022-09-26T07:44:39+0000,P988js8z2nk
4,3,"""기사를 쓰는거냐? 선동을 하는거냐? 개나소나 기자라고... 저건 팩트를 전달하겠다...",2,"""2xLak""",7,10,"""2013-08-31T11:09:29+0900""",12163941,12163941,102,2022-09-26T07:44:39+0000,P988js8z2nk
...,...,...,...,...,...,...,...,...,...,...,...,...
1175797,13,"""코로나 백신 페기도 기사화 해주시죠..""",0,"""3CxLG""",0,0,"""2022-08-31T10:20:55+0900""",767965981585703361,767965981585703361,102,2022-09-26T13:58:34+0000,Vm2WPiKsIw0
1175798,14,"""그래 좋은 정책👍""",0,"""2k0ZP""",1,2,"""2022-08-31T10:15:56+0900""",767965660922773957,767965660922773957,102,2022-09-26T13:58:34+0000,Vm2WPiKsIw0
1175799,15,"""정신나간 개돼지들이 추석연휴때 또 줄서서 PCR 유르르 받겠네.""",1,"""1kS72""",0,1,"""2022-08-31T09:57:01+0900""",767964442628456458,767964442628456458,102,2022-09-26T13:58:34+0000,Vm2WPiKsIw0
1175800,16,"""백신팔이 가동한 윤깡통을 지지하는 대깨윤들은 정부에서 4분기에 준다는 개량백신 다...",0,"""100n7""",3,5,"""2022-08-31T09:59:27+0900""",767964598623011043,767964598623011043,102,2022-09-26T13:58:34+0000,Vm2WPiKsIw0


In [166]:
sum(total_df['cid'] == total_df['pid'])

1175802

In [171]:
total_df.reset_index(drop = True, inplace = True)

In [5]:
lst = []
TARGET = '포스코'


for idx, i in enumerate(total_df['comment']):
    if TARGET in str(i):
        lst.append(idx)

list(set(lst))

total_df.loc[lst,'comment']

17260      "하긴 경부선이나 대부분의 철도는 일본이 깐거 유지보수하면서 지켜왔는데 저걸 언제또...
17280      "하긴 경부선이나 대부분의 철도는 일본이 깐거 유지보수하면서 지켜왔는데 저걸 언제또...
17300      "하긴 경부선이나 대부분의 철도는 일본이 깐거 유지보수하면서 지켜왔는데 저걸 언제또...
17320      "하긴 경부선이나 대부분의 철도는 일본이 깐거 유지보수하면서 지켜왔는데 저걸 언제또...
17340      "하긴 경부선이나 대부분의 철도는 일본이 깐거 유지보수하면서 지켜왔는데 저걸 언제또...
                                 ...                        
1094275    "준비가 너무 미흡한 사전점검이었습니다. 기본적 것도 미시공인데 무얼 점검합니까? ...
1095072    "포스코건설...이게 말이됩니까?\n저렇게 만들어놓고 사전점검을 하는곳이 어디있답니...
1095076                               "포스코건설 이건 너무하자나요!!!!!"
1108172    "지난번에 포스코 하고 합작해서 더 우수한 가스 탱크를 개발해서 관련업계에서 곧 상...
1128887    "정유사와 석유회사 구분도 못하는 ㅁㅈㄹ들이 넘쳐흐르네... 석유회사는 유정이 주 ...
Name: comment, Length: 184, dtype: object